# Functions for the Resistance Subpopulation Model

In [4]:
#import packages
import pandas as pd
import numpy as np
import random as rd
from scipy.integrate import odeint
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Time domain
tmin,tmax = 0,8
delta = 0.001
nsteps = (tmax-tmin)/delta
t = np.linspace(tmin,tmax,int(nsteps))

#parameters
mu = .36 
phi = 6e-08 
lamb = 0.65 
beta = 90 
Pr_S = 0.60 


#Define parameters for resistant host supopulation
muR = .44
phiR = 3e-9
lambR = 1,1,4,3.5
betaR = 60,60,150,180
Pr_R = np.round(np.arange(0, 1.1,.1),2) 

R0 = 300000,250000,200000,150000,100000,50000,10000
R0_19 = 460000,400000,340000,280000,220000,160000,100000


def RSM(y,t,mu,phi,beta,lamb,phiR,betaR,lambR,Pr_S,Pr_R):
    S,I,V,R,IR = y
    dSdt = (mu*S) - (phi*S*V) + (lamb*(I)*(1-Pr_S))
    dIdt = (phi*S*V) - (lamb*I)
    dRdt = (mu*R) - (phiR*R*V) + (lambR*(IR)*(1-Pr_R))
    dIRdt = (phiR*R*V) - (lambR*IR)
    dVdt = ((beta*I*lamb*Pr_S) + (betaR*IR*lambR*Pr_R)) - ((phi*S+phiR*R)*V)
    return np.array([dSdt,dIdt,dVdt,dRdt,dIRdt])

def solver(model,mu,phi,beta,lamb,phiR,betaR,lambR,Pr_S,PR_R):
    Ss,Is,Vs,Rr,Ir = [],[],[],[],[]
    ODE = np.array([odeint(RSM, model, t, args=(mu,phi,beta,lamb,phiR,betaR,lambR,Pr_S,PR_R[i])) for i in range(len(Pr_R))])
    pd.Series([Ss.append(ODE[i,:,0]) for i in range(len(Pr_R))])
    pd.Series([Is.append(ODE[i,:,1]) for i in range(len(Pr_R))])
    pd.Series([Vs.append(ODE[i,:,2]) for i in range(len(Pr_R))])
    pd.Series([Rr.append(ODE[i,:,3]) for i in range(len(Pr_R))])
    pd.Series([Ir.append(ODE[i,:,4]) for i in range(len(Pr_R))])
    total = [pd.Series((Ss[i]+Is[i]+Rr[i]+Ir[i])) for i in range(len(Pr_R))]
    viruses = [pd.Series(Vs[i]) for i in range(len(Pr_R))]
    ode_df = pd.DataFrame(total)
    ode_df_transpose= ode_df.transpose()
    ode_df_hostlog = np.log10(ode_df_transpose)
    viruses_df = pd.DataFrame(viruses)
    viruses_df_transpose = viruses_df.transpose()
    ode_df_VLPlog = np.log10( viruses_df_transpose)
    return ode_df_hostlog,ode_df_VLPlog

def solverR(model,mu,phi,beta,lamb,phiR,betaR,lambR,Pr_S,PR_R):
    Ss,Is,Vs,Rr,Ir = [],[],[],[],[]
    ODE = np.array([odeint(RSM, model, t, args=(mu,phi,beta,lamb,phiR,betaR,lambR,Pr_S,PR_R[i])) for i in range(len(Pr_R))])
    pd.Series([Ss.append(ODE[i,:,0]) for i in range(len(Pr_R))])
    pd.Series([Is.append(ODE[i,:,1]) for i in range(len(Pr_R))])
    pd.Series([Vs.append(ODE[i,:,2]) for i in range(len(Pr_R))])
    pd.Series([Rr.append(ODE[i,:,3]) for i in range(len(Pr_R))])
    pd.Series([Ir.append(ODE[i,:,4]) for i in range(len(Pr_R))])
    total = [pd.Series((Ss[i]+Is[i]+Rr[i]+Ir[i])) for i in range(len(Pr_R))]
    viruses = [pd.Series(Vs[i]) for i in range(len(Pr_R))]
    ode_df = pd.DataFrame(total)
    ode_df_transpose= ode_df.transpose()
    ode_df_hostlog = np.log10(ode_df_transpose)
    viruses_df = pd.DataFrame(viruses)
    viruses_df_transpose = viruses_df.transpose()
    ode_df_VLPlog = np.log10( viruses_df_transpose)
    return ode_df_hostlog,ode_df_VLPlog


from math import e
def combined_variance(host,VLPS, model_host, model_vlp,file_host,file_vlp):
    SST_host,RSS_host = [],[]
    SST_VLP, RSS_VLP = [],[]
    indices_host = np.r_[[np.where(min(abs(t-d))==abs(t-d))[0][0]for d in file_host.Day]] #index model for host
    model_data_host = np.array(model_host[indices_host]) #find host concentrations associated with index values
    indices_vlp = np.r_[[np.where(min(abs(t-d))==abs(t-d))[0][0]for d in file_vlp.Day]] #index model for VLP
    model_data_vlp = np.array(model_vlp[indices_vlp]) #find VLP concentrations associated with index values
    y_actual_mean_host = np.sum(host)/len(host) #calculate mean for host
    [SST_host.append((host[i]-y_actual_mean_host)**2) for i in range(9)]  
    [RSS_host.append((host[i]-model_data_host[i])**2) for i in range(9)]
    r_squared_host = 1-(sum(RSS_host))/(sum(SST_host)) #normal R-squared for host
    y_actual_mean_VLP = np.sum(VLPS)/len(VLPS)   #calculate mean for VLP
    [SST_VLP.append((VLPS[i]-y_actual_mean_VLP)**2) for i in range(3)]
    [RSS_VLP.append((VLPS[i]-model_data_vlp[i])**2) for i in range(3)]
    r_squared_VLP = 1-(sum(RSS_VLP))/(sum(SST_VLP)) #normal R-squared for host
    sigma = 0.2
    Likelihood_host = e**(-((1/sigma)*sum(RSS_host))) #Likelihood for host
    Likelihood_VLP = e**(-((1/sigma)*sum(RSS_VLP))) #Likelihood for VLP
    Likelihood_combined = e**(-((1/sigma)*sum(RSS_VLP+RSS_host))) #combined likelihood
    adjusted_r_squared_host = 1 - (((1-r_squared_host)*(9-1))/(9-2-1)) #adjusted R-squared for host
    return Likelihood_combined

def orientdata26(data):
    likelihood26 = []
    likelihood26 = np.array([(data[i]['Likelihood']) for i in range(7)])
    likelihood_df26 = pd.DataFrame(likelihood26, columns=[str(i) for i in Pr_R], index= [str(i) for i in R0])
    return likelihood_df26

def orientdata19(data):
    likelihood19 = []
    likelihood19 = np.array([(data[i]['Likelihood']) for i in range(7)])
    likelihood_df19 = pd.DataFrame(likelihood19, columns=[str(i) for i in Pr_R], index= [str(i) for i in R0_19])
    return likelihood_df19

def maxvalue(dataframe):
    maxvalues = []
    maxvalues = dataframe.max()
    finalmax = maxvalues.max()
    maxValueIndex = maxvalues.idxmax()
    rounded_finalmax = round(finalmax, 2)
    return rounded_finalmax, maxValueIndex
